In [1]:
import numpy as np
import matplotlib.pyplot as plt
import pandas as pd

In [ ]:
def shuffle_examples(X,y):
  p = np.random.permutation(len(X))
  return X[p],y[p]

def split_train_validation_test(X,y,p_t,p_tv):
  n = int(p_t*len(X))
  m = int(p_tv*len(X))
  return X[:n],y[:n],X[n:m],y[n:m],X[m:],y[m:]   

def scale(x):
  mu = np.mean(x,axis=0)
  st = np.std(x,axis=0)
  return (x-mu)/st,mu,st  

def sigmoid(x):
  return 1/(1+np.exp(-x))

def relu(x):
  return np.maximum(0,x)  

def softmax(X):
  return np.exp(X)/(np.sum(np.exp(X),axis=1).reshape(-1,1))   

def sigmoid_p(x):
  return sigmoid(x)*(1-sigmoid(x))

def relu_p(x):
  return 1*(x >= 0)    

def error(y,y_hat):
  return -np.sum(y*np.log(y_hat)+(1-y)*np.log(1-y_hat))/len(y)

def error_softmax(y,y_hat):
  return -np.sum(y*np.log(y_hat))/len(y)
  
def error_reg(y,y_hat,W,la):
  J_reg = -np.sum(y*np.log(y_hat)+(1-y)*np.log(1-y_hat))/len(y)
  for l in range(1,len(W)):
    J_reg = J_reg + la*np.sum(W[l]**2)/len(y)
  return J_reg      

def error_softmax_reg(y,y_hat,W,la):
  J_reg = -np.sum(y*np.log(y_hat))/len(y)
  for l in range(1,len(W)):
    J_reg = J_reg + la*np.sum(W[l]**2)/len(y)
  return J_reg   
  
def accuracy_recall_precision(y_hat,y):
  y_hat_cat = 1*(y_hat > 0.5)
  correct_predictions = np.sum(y_hat_cat == y)
  predictions = len(y)
  true_positives = np.sum((y_hat_cat == 1) & (y == 1))
  positives = np.sum(y)
  predicted_positives = np.sum(y_hat_cat)
  accuracy = correct_predictions/predictions
  recall = true_positives/positives
  precision = true_positives/predicted_positives
  return accuracy, recall, precision  

# y is a one dimensional array y_1d = [0,2,1,3,0,2] (the categories of the examples)
def accuracy_softmax(y_hat,y_1d):
  y_hat_1d = np.argmax(y_hat,axis=1)
  correct_predictions = np.sum(y_hat_1d == y_1d)
  predictions = len(y)
  accuracy = correct_predictions/predictions
  return accuracy
  
def h(x,act):
  if act == 'sigmoid':
    return sigmoid(x)
  if act == 'relu':
    return relu(x)
  if act == 'identity':
    return x
  if act == 'tanh':
    return np.tanh(x)
  if act == 'softmax':
    return softmax(x)  
  return 'Problem' 

def h_p(x,act):
  if act == 'sigmoid':
    return sigmoid_p(x)
  if act == 'relu':
    return relu_p(x)
  if act == 'identity':
    return 1
  if act == 'tanh':
    return 1/(np.cosh(x))**2
  return 'Problem'  

def As_Zs(X,W,b,act):
  A = [X]
  Z = [0]
  for l in range(1,len(b)):
    Z.append(np.matmul(A[-1],W[l])+b[l])
    A.append(h(Z[-1],act[l]))
  return A,Z 

def gradients(A,Z,act,W,y):
  y_hat = A[-1]
  l = len(W)-1
  DJ_DW = []
  DJ_Db = []
  DJ_DZ = (y_hat - y.reshape(-1,1))/len(y)
  while l > 1:
    DJ_DA = np.matmul(DJ_DZ,W[l].T)
    DJ_DW.insert(0,np.matmul(A[l-1].T,DJ_DZ)) 
    DJ_Db.insert(0,np.sum(DJ_DZ,axis=0))
    DJ_DZ = DJ_DA*h_p(Z[l-1],act[l-1])
    l = l-1
  DJ_DA = np.matmul(DJ_DZ,W[1].T)
  DJ_DW.insert(0,np.matmul(A[0].T,DJ_DZ)) 
  DJ_Db.insert(0,np.sum(DJ_DZ,axis=0)) 
  DJ_DW.insert(0,0) 
  DJ_Db.insert(0,0)  
  return DJ_DW,DJ_Db  

def gradients_softmax(A,Z,act,W,y):
  y_hat = A[-1]
  l = len(W)-1
  DJ_DW = []
  DJ_Db = []
  DJ_DZ = (y_hat - y)/len(y)
  while l > 1:
    DJ_DA = np.matmul(DJ_DZ,W[l].T)
    DJ_DW.insert(0,np.matmul(A[l-1].T,DJ_DZ)) 
    DJ_Db.insert(0,np.sum(DJ_DZ,axis=0))
    DJ_DZ = DJ_DA*h_p(Z[l-1],act[l-1])
    l = l-1
  DJ_DA = np.matmul(DJ_DZ,W[1].T)
  DJ_DW.insert(0,np.matmul(A[0].T,DJ_DZ)) 
  DJ_Db.insert(0,np.sum(DJ_DZ,axis=0)) 
  DJ_DW.insert(0,0) 
  DJ_Db.insert(0,0)  
  return DJ_DW,DJ_Db  

def update_parameters(W,b,DJ_DW,DJ_Db,c):
  for l in range(1,len(b)):
    W[l] = W[l] - c*DJ_DW[l]
    b[l] = b[l] - c*DJ_Db[l]
  return W, b  

def update_parameters_reg(W,b,DJ_DW,DJ_Db,c,m,la):
  for l in range(1,len(b)):
    W[l] = W[l] - c*DJ_DW[l] - 2*c*la*W[l]/m
    b[l] = b[l] - c*DJ_Db[l] 
  return W, b  

def initialize_W_and_b(n):
  W = [0]
  b = [0]
  for l in range(1,len(n)):
    W.append(np.random.randn(n[l-1],n[l])/np.sqrt(n[l-1]))
    b.append(np.zeros(n[l]))
  return W, b 

def steepest(n,act,X,y,epochs,c):
  W, b = initialize_W_and_b(n)
  J_list = []
  for i in range(epochs):
    A, Z = As_Zs(X,W,b,act)
    y_hat_2d = A[-1]
    J_list.append(error(y,y_hat.reshape(-1)))
    DJ_DW, DJ_Db = gradients(A,Z,act,W,y)
    W, b = update_parameters(W,b,DJ_DW,DJ_Db,c)
  return W, b, J_list   

def steepest_softmax(n,act,X,y,epochs,c):
  W, b = initialize_W_and_b(n)
  J_list = []
  for i in range(epochs):
    A, Z = As_Zs(X,W,b,act)
    y_hat = A[-1]
    J_list.append(error_softmax(y,y_hat))
    DJ_DW, DJ_Db = gradients_softmax(A,Z,act,W,y)
    W, b = update_parameters(W,b,DJ_DW,DJ_Db,c)
  return W, b, J_list   
  
def steepest_reg(n,act,X,y,epochs,c,la):
  W, b = initialize_W_and_b(n)
  J_list = []
  for i in range(epochs):
    A, Z = As_Zs(X,W,b,act)
    y_hat = A[-1]
    J_list.append(error_reg(y,y_hat.reshape(-1),W,la))
    DJ_DW, DJ_Db = gradients(A,Z,act,W,y)
    W, b = update_parameters_reg(W,b,DJ_DW,DJ_Db,c,len(y),la)
  return W, b, J_list           

def steepest_softmax_reg(n,act,X,y,epochs,c,la):
  W, b = initialize_W_and_b(n)
  J_list = []
  for i in range(epochs):
    A, Z = As_Zs(X,W,b,act)
    y_hat = A[-1]
    J_list.append(error_softmax_reg(y,y_hat,W,la))
    DJ_DW, DJ_Db = gradients_softmax(A,Z,act,W,y)
    W, b = update_parameters_reg(W,b,DJ_DW,DJ_Db,c,len(y),la)
  return W, b, J_list           

def predict(X,W,b,act):
  A,Z = As_Zs(X,W,b,act)
  return A[-1]

def scale_predict(X,X_train_mean,X_train_std,W,b,act):
  return predict((X-X_train_mean)/X_train_std,W,b,act)

In [2]:
# input: y, the labels of the examples, and s, the number of categories. y is 1d
# Ex y = [0,2,1] s = 3
# output + [[1,0,0]
#           [0,0,1]
#           [0,1,0]]

def labels_1d_2d(y,n_c):
  return 1*(np.arange(n_c).reshape(1,-1) == y.reshape(-1,1))   